In [1]:
#User-defined inputs and assumptions:

rf = 0.03      #Annualized risk free rate assumption
start = 1000   #The inital asset value for wealth index
floor = 0.8    #CPPI floor = 80% of the initial asset value (represents a 20% drawdown constraint)

In [2]:
import edhec_risk_kit as erk
import pandas as pd
import numpy as np

ind_return = erk.get_ind_returns()
tmi_return = erk.get_total_market_index_returns()

#Define "Risky" Assets
risky_r = ind_return["2000":][["Steel","Fin","Beer"]]

#Set up a Data Frame for the "Safe" Asset
safe_r = pd.DataFrame().reindex_like(risky_r) #gives you an empty dataframe of the same shape as risky_r
safe_r[:] = rf/12

In [3]:
#Make sure that risky_r.shape = safe_r.shape
risky_r.shape == safe_r.shape

True

Objective: write an algorithm for implementing CPPI and Drawdown Constraints (Dynamic Hedging).

For every unit of time:
1. Cushion = (Asset Value) minus (Floor Value): This is your risk budget
2. Compute an Allocation to Safe and Risky Asset = M(t)*Cushion
3. Recompute the Asset Value based on the returns

#### Side note: you can use numpy's buit-in functions to improve your code:

- It's good practice to avoid using loops with numpy because they require a lot more processing power than numpy functions.

In [4]:

def compound1(r):
    return np.exp(np.log(1+r).sum())-1 #np.exp()-1 and log(1+r) both represent unnecessary loops
%timeit compound1(risky_r) #the processing power it takes to compound returns using compound1()

def compound2(r):
    return np.exp(np.log1p(r).sum())-1 #np.log1p(r) is mathematically equivalent to np.log(1+r)
%timeit compound2(risky_r) #the processing power it takes to compound returns using compound2()
      
def compound3(r):
    return np.expm1(np.log1p(r).sum()) #np.expm1(r) is mathematically equivalent to exp(r)-1
%timeit compound3(risky_r) #the processing power it takes to compound returns using compound3()

def compoundBASE(r):
    return (1+r).prod()-1
%timeit compoundBASE(risky_r) #the processing power it takes to compound returns using base python

print("On average, numpy compound3() beats all other options in terms of processing efficiency just by making good use of numpy's built-in functions log1p(r) and expm1()")
print("But in this case, it's comparable to using base python to calculate compounded returns...")

541 µs ± 3.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
473 µs ± 10.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
385 µs ± 751 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


NameError: name 'compound4' is not defined

#### Now back to CPPI:

- Here, it's unavoidable... we must use a for() loop to run the algorithm for CPPI

In [ ]:
dates = risky_r.index
n_steps = len(dates)
account_value = start #initailize account_value
floor_value = start*floor #initailize floor_value
m = 3
account_history = pd.DataFrame().reindex_like(risky_r)
cushion_history = pd.DataFrame().reindex_like(risky_r)
risky_w_history = pd.DataFrame().reindex_like(risky_r)

#Implement the CPPI algorithim with a for() loop
for step in range(n_steps):
    cushion = (account_value - floor_value)/account_value
    risky_w = m*cushion
    risky_w = np.minimum(risky_w, 1) #constraint: risky_w cannot exceed 100% i.e., we do not allow levered CPPI investment (must borrow to finance risky_w > 100%)
    risky_w = np.maximum(risky_w, 0) #constraint: cannot short the risky asset
    safe_w = 1 - risky_w
    risky_alloc = account_value*risky_w
    safe_alloc = account_value*safe_w
    ## update the account value for this time step
    account_value = risky_alloc*(1+risky_r.iloc[step]) + safe_alloc*(1+safe_r.iloc[step])
    ## populate account_history, cushion_history, risky_w_history
    account_history.iloc[step] = account_value
    cushion_history.iloc[step] = cushion
    risky_w_history.iloc[step] = risky_w
    

In [ ]:
#Let's look at the growth of $1000 for a 100% allocaiton to each of our risky assets
risky_wealth = start*(1+risky_r).cumprod() #your compounded account value lacking any CPPI strategy
risky_wealth.plot(figsize=(12,6))

In [ ]:
#Let's look at the value added by CPPI if our risky asset is an investment in the steel industry
ax = account_history["Steel"].plot(figsize=(12,6), legend=True, label="CPPI Strategy")
risky_wealth["Steel"].plot(ax=ax,figsize=(12,6), style="k:", legend=True, label="100% Invested in Risky Asset", title="Growth of $1000")
ax.axhline(y=floor_value, color="red", linestyle="--")

In [ ]:
#Let's look at the allocation to our risky asset over history
risky_w_history["Steel"].plot(figsize=(12,6), title="Historic Portfolio %Weight in Risky Asset as per CPPI Strategy")